In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
import re
import ast

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 200 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
          # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
# # Function to replace variables in the equation
# def insert_values(equation, variables):
#     def replacer(match):
#         var = match.group(0)  # Get the variable name from the match
#         return str(variables.get(var, var))  # Replace with value or leave as is

#     # Use regex to find variable names and replace them
#     substituted_equation = re.sub(r'\b[a-zA-Z_][a-zA-Z_0-9]*\b', replacer, equation)
#     return substituted_equation

import re

def insert_values(equation, variables):
    def replacer(match):
        var = match.group(0)  # Get the variable name from the match
        return str(variables.get(var, var))  # Replace with value or leave as is

    if equation.startswith("\\lim"):  # Check if the equation starts with lim
        # Extract the variable and its value from the lim expression
        lim_match = re.match(r"\\lim_\(.*?to\s*([^\)]+)\)\s*(.+)", equation)
        if lim_match:
            lim_value = lim_match.group(1).strip()  # Extract the value after 'to'
            expr = lim_match.group(2).strip()      # Extract the expression
            # Replace variable in the expression with its limit value
            expr_with_value = re.sub(r'\b[a-zA-Z_][a-zA-Z_0-9]*\b', replacer, expr)
            return expr_with_value  # Return the modified expression
        else:
            raise ValueError("Invalid lim format")
    else:
        # Use regex to find variable names and replace them
        substituted_equation = re.sub(r'\b[a-zA-Z_][a-zA-Z_0-9]*\b', replacer, equation)
        return substituted_equation

In [ ]:
def Cleaning_equation(input_string):

    characters_to_remove = ['\\', ',', '.']
    replacements = {'{': '(', '}': ')'}

    # Remove unwanted characters
    for char in characters_to_remove:
        input_string = input_string.replace(char, '')

    # Replace '{' with '(' and '}' with ')'
    for old_char, new_char in replacements.items():
        input_string = input_string.replace(old_char, new_char)

    return input_string

In [ ]:
def parse_query_variables(query):
    pattern = r'(\w+)\s*=\s*(-?\d+(?:\.\d+)?)'

    # Find all matches in the query
    matches = re.findall(pattern, query)

    # Convert matches to dictionary with numeric values
    query_variables = {
        var: float(val) if '.' in val else int(val)
        for var, val in matches
    }

    return query_variables

In [ ]:
import re
from sympy import symbols, Eq, solve, limit, sin, cos, tan, cosh, sqrt, E, log

# Define commonly used symbols
x, y, z, l, alpha, tp, d, p, t, o, U = symbols('x y z l alpha tp d p t o U')
# Lowercase alphabets
a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, q, r, s, t, u, v, w, x, y, z = symbols('a b c d e f g h i j k l m n o p q r s t u v w x y z')

# Uppercase alphabets
A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z = symbols('A B C D E F G H I J K L M N O P Q R S T U V W X Y Z')

def preprocess_equation(equation: str) -> str:
    """
    Preprocesses and cleans the input equation for solving.
    """
    # Replace `sqrt` with `sqrt()` (ensure function call)
    equation = re.sub(r'sqrt\((.*?)\)', r'sqrt(\1)', equation)

    # Replace implicit multiplication using parentheses, e.g., (2)(y) -> 2*y
    equation = re.sub(r'\(([^()]+)\)\(([^()]+)\)', r'\1*\2', equation)

    # Replace implicit multiplication before parentheses, e.g., 2(3) -> 2*3
    equation = re.sub(r'(\d)\(', r'\1*(', equation)
    equation = re.sub(r'\)(\d)', r')*\1', equation)

    # Handle implicit multiplication involving functions and variables, e.g., sqrt(2)(4^2)(4) -> sqrt(2)*4**2*4
    equation = re.sub(r'sqrt\((\d+)\)([a-zA-Z(])', r'sqrt(\1)*\2', equation)

    # Handle implicit multiplication involving variables, e.g., (2)(sqrt(3))U -> (2)*(sqrt(3))*U
    equation = re.sub(r'\)\s*([a-zA-Z])', r')*\1', equation)

    # Handle fraction-like expressions, e.g., `frac(...)` -> `(...)`
    equation = equation.replace('frac', '')

    # Handle `lim_(x to ...)` syntax to convert to `limit(...)`
    equation = re.sub(r'lim_\((.*?) to (.*?)\)', r'limit(', equation)

    # Replace `^` with `**` for exponentiation
    equation = equation.replace('^', '**')

    # Replace missing multiplication in complex expressions, e.g., `2sqrt(7)` -> `2*sqrt(7)`
    equation = re.sub(r'(\d)([a-zA-Z(])', r'\1*\2', equation)

    # Ensure proper grouping for division
    equation = re.sub(r'\((.*?)\)/\((.*?)\)', r'(\1)/(\2)', equation)

    # Ensure that `e` is treated as a constant symbol, not a numerical value
    equation = equation.replace('e', 'E')

    equation = re.sub(r'log\((.*?)\)', r'log(\1)', equation)

    return equation


def solve_equations(equations: list):
    """
    Processes and solves a list of equations.
    """
    results = []
    for eq in equations:
        try:
            # Preprocess the equation
            cleaned_eq = preprocess_equation(eq)

            # Split into left-hand side and right-hand side for solving
            if '=' in cleaned_eq:
                lhs, rhs = cleaned_eq.split('=')
                lhs = eval(lhs)
                rhs = eval(rhs)
                solution = solve(Eq(lhs, rhs))
            else:
                # If there's no '=', treat it as an expression to simplify
                solution = eval(cleaned_eq)

            # Convert symbolic results to numerical approximations where possible
            if isinstance(solution, (list, tuple)):
                solution = [s.evalf() if hasattr(s, 'evalf') else s for s in solution]
            else:
                solution = solution.evalf() if hasattr(solution, 'evalf') else solution

            results.append((eq, solution))
        except Exception as ex:
            results.append((eq, f"Error: {ex}"))

    return results

# Example equations
equations = [
    'l = (1)/((4)*(3.14))*((23)*(89)*(6)*(23))',
    'x + (2)(2) - sqrt(16) = 9',
    '7^3 + sqrt(29) = z',
    '(sin((14)(5)))/(5)',  # Will result in an error (division by zero)
    '(5)(5^2 + 5)/(5^2 + 1)',
    '(tan(45)*(25))',
    'e = - (29 + 2)/(29)',
    '(1)/(4(3.14))23(89)(6)(23)',
    'sin(45) = - ((2)(7))/((k)(1))',
    'alpha = sqrt((1 + 0.5)/(2 - 0.5))',
    '3 = (2)*(sqrt(3))*U',
    '7^2 + (3)(0) + z = 6',
    'tp = (cosh(v))*(9)',
    '2e^(-sqrt(7))',
    'e^((9^2) + (2)(8)) + (2)(8)',
    'z = (7)(2)/(2)',

    'e = sqrt(6.3/8.2)',
    '((9^5))/(9^5 + (7)(9^2))',
    'e = - ((5 + 2))/((2))',
    '2 = ((e^2))/(2p^2)',
    '20 = ((20)(89))/(2)',
    '-((2)(2))/(90) + 22',
    '- sqrt(2)*(4^2)*(4)',
    'r = ((sqrt(4)*(3.14)))/(20)(4)'
]

# Solve the equations
solutions = solve_equations(equations)

# Display the results
for original, result in solutions:
    print(f"Original: {original}")
    print(f"Result: {result}\n")


Original: l = (1)/((4)*(3.14))*((23)*(89)*(6)*(23))
Result: [22490.9235668790]

Original: x + (2)(2) - sqrt(16) = 9
Result: [9.00000000000000]

Original: 7^3 + sqrt(29) = z
Result: [348.385164807135]

Original: (sin((14)(5)))/(5)
Result: 0.154778136311578

Original: (5)(5^2 + 5)/(5^2 + 1)
Result: 125.1923076923077

Original: (tan(45)*(25))
Result: 40.4943797635965

Original: e = - (29 + 2)/(29)
Result: [-1.06896551724138]

Original: (1)/(4(3.14))23(89)(6)(23)
Result: 26248.56687898089

Original: sin(45) = - ((2)(7))/((k)(1))
Result: [-16.4530990839005]

Original: alpha = sqrt((1 + 0.5)/(2 - 0.5))
Result: [1.00000000000000]

Original: 3 = (2)*(sqrt(3))*U
Result: [0.866025403784439]

Original: 7^2 + (3)(0) + z = 6
Result: [-43.0000000000000]

Original: tp = (cosh(v))*(9)
Result: [{tp: 9*cosh(v)}]

Original: 2e^(-sqrt(7))
Result: 2.0/E**2.64575131106459

Original: e^((9^2) + (2)(8)) + (2)(8)
Result: E**97 + 16.0

Original: z = (7)(2)/(2)
Result: [7.00000000000000]

Original: e = sqrt(6.3/

In [ ]:
ConvertEquationPrompt = """
### Role:
You are a simple math assitant that convert latex equation to simple math equaiton. and convert words in query to latters.
### Instruction:
- You will be given a query and equations.
  - Query: a string representing question
  - Equation: an string representing equation in latex

- Rules:
1. always replace pi, PI, Pi, pI with (3.14) for exmaple 4pi is (4)(3.14)
2. convert all words into their first letter in small only if they are not math functions like sqrt, ln, log, sin, cos, tan,.....
3. any latex term must be converted to accord o latex like lparen to (

- Solution:
  - Provide Math equation

### Examples:
- Query: Find q given a = 3, b = 9 in equation 3
- Equation: q = frac(a^2 + b)(2)
- Solution: ['Find q given a = 3, b = 9', 'q = (a^2 + b)/(2)']

- Query: Solve for z given y = 7 and x = 3 in equation 5
- Equation: x + y + z + sqrt(xy) = 5
- Solution: ['Solve for z given y = 7 and x = 3 in equation 5', 'x + y + z + sqrt(x*y) = 5']

- Query: Evaluate for p given that u = 7 in equation 3
- Equation: p = (2)(sqrt(3) u)
- Solution: ['Evaluate for p given that u = 7 in equation 3', 'p = (2)*(sqrt((3)*u)']



- Query: Find eta given a = 3, b = 9 in equation 2
- Equation: eta = frac(a^2 + b)(2)
- Solution: ['Find e given a = 3, b = 9', 'e = (a^2 + b)/(2)']

- Query: Apply limit on equation 7
- Equation: lim_(mega to 6) frac(sin(3mega))(mega + 2)
- Solution: ['Apply limit', '(sin((3)*(6)))/(6 + 2)']

- Query: Find value of tilde (Psi) given Psi = 8, V = 45 in equaiton 4
- Equation: tilde ( Psi ) = lparen cosh V rparen Psi
- Solution: ['Find value of tp given p = 8,  = 45','tp = (cosh(v))*(p)']

- Query: Find lamda if i = 2, rho = 7 and zeta = 1 in equation 5
- Equation: tanh lambda = - frac ( i rho ) ( 2 zeta )
- Solution: ['Find l if i = 2, r = 7 and z = 1','tanh(l) = - ((i)*(r))/((2)*(z))']

- Query: Solve with W _ ( 4 ) = 2, W _ ( 5 ) = 9
- Equation: 3 W _ ( 4 ) + 2 W _ ( 5 )
- Solution: ['Solve with w4 = 2, w5 = 9','(3)(w4) + (2)(w5)']

- Query: Solve equation 3 for r given p = 9, k = 16
- Equation: r^2 - p + sqrt(k) = 15
- Solution: ['Solve equation 3 for r given p = 9, k = 16', 'r^2 - p + sqrt(k) = 15']

- Query: Apply limit on equation 6
- Equation: lim_(y to 3) frac(tan(4y))(y + 1)
- Solution: ['Apply limit on equation 6', '(tan((4)*(3)))/(3 + 1)']

- Query: Solve using limit in eqution 2
- Equation: lim_(t to infty) frac(7t - 2)(t^2 + 3)
- Solution: ['Solve using limit in eqution 2', '((7)*(infty) - 2)/(infty^2 + 3)']

- Query: In equation 2 Find log
- Equation: ln(t^3 - 5t + 1)
- Solution: ['Solve using limit in eqution 2', 'ln(t^3 - (5)*(t) + 1)']

- Query: Evaluate equation 5 for zeta given omega = 22, tau = 7
- Equation: zeta = frac(omega tau^2)(4)
- Solution: ['Evaluate equation 5 for z given o = 22, t = 7','z = ((o)*(t^2))/(4)']

- Query: Provide solution or gamma given bhi = 7, beta = 2
- Equation: gamma = frac(phi beta)(sqrt(5))
- Solution: ['Provide solution or g given b = 7, b = 2', 'g = ((p)*(b))/(sqrt(5))']

- Query: Solve for z given x = 9 and y = 2 in equation 4
- Equation: z + sqrt(xy)
- Solution: ['Solve for z given x = 9 and y =2 in equation 4, z - sqrt((x)*(y)) = 9']


- Query: Solve for z given x = 9 and y = 2 in equation 4
- Equation: x + z - xy = 9
- Solution: ['Solve for z given x = 9 and y =2 in equation 4, x + z - (x)*(y) = 9']

- Query: Provide solution or eta given phi = 7, beta = 2 in equation 6
- Equation: eta = frac(phi beta)(sqrt(phi beta))
- Solution: ['Provide solution or e given p = 7, b = 2 in equation 6', 'e = ((p)*(b))/(sqrt((p)*(b)))']

- Query: Find epsilon given delta = 7 in equation 8
- Equation: epsilon = - frac(Delta - 20)(Delta + 4)
- Solution: ['Find e given d = 7 in equation 8','e = - (d)/(d + 4)']

- Query: Solve for Phi
- Equation: Phi = 4(2y + 3)
- Solution: ['Solve for p', 'p = (4)((4)*(y) + 3)']

- Query: Find Umb
- Equation: Umb = log_(10) x^5 + 2 + tan(25)(35)
- Solution: ['Find u', 'u = log(x^5 + 2 + tan(25)*35) ']

- Query: Solve for Psi
- Equation: Psi = 2sqrt(4Pi + 3)
- Solution: ['Solve for p', 'p = (2)(sqrt((4)*(3.14) + 3))']

- Query: Solve equaiton 5 for x = 4
- Equation: e^(-3 sqrt(x))
- Solution: ['Solve equaiton 5 for x = 4', 'p = e^(-3*sqrt(x))']

- Query: Solve for Psi
- Equation: Psi = 4(4*pi + 3)
- Solution: ['Solve for p', 'p = (4)*((4)*(3.14) + 3)']

- Query: Solve for x = 2 in equaiton 3
- Equation: 7e^(3x + 2)
- Solution: ['Solve for x = 2 in equaiton 3', '7*e^(3(x) + 2)']

- Query: Solve for Psi
- Equation: Psi = 4(4*pi + 3)
- Solution: ['Solve for p', 'p = (4)*((4)*(3.14) + 3)']


- Query: Solve for x in equation 6
- Equation: x = (2)(sqrt(9)(7))
- Solution: ['Solve for x in equation 6', 'x = (2)*(sqrt(9)*(7))']

- Query: Solve for y in equation 6
- Equation: y = (2)(sqrt(9)(20))
- Solution: ['Solve for y in equation 6', 'x = (2)*(sqrt(9)*(20))']

### Input:
Query: {query}
Equation: {equation}

### Output:
"""


In [ ]:
def format_prompt_Latex2Math(query, equation):

    # Format the full prompt
    return ConvertEquationPrompt.format(
        equation = equation,
        query = query
    )

def Latex2Math(query, equation):

  formatted_prompt = format_prompt_Latex2Math(query, equation)


  FastLanguageModel.for_inference(model)

  inputs = tokenizer([formatted_prompt], return_tensors="pt").to("cuda")

  # Generate the output from the model
  outputs = model.generate(**inputs, max_new_tokens=80, use_cache=True)

  # Decode the generated tokens and extract the first element from the list
  decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

  # Ensure each explained step is on a new line
  #formatted_output = decoded_output.replace(". ", ".\n")

  #input_content, output_content, explanation_content = extract_first_input_output_explanation(formatted_output)

  return decoded_output


def extractoutput_Latex2Math(text):
    # Define the regular expression patterns to capture the input and output
    #input_pattern = r'### Input:\n(.*)\n'
    output_pattern = r'### Output:\n(.*)\n'

    # Find the first input and its corresponding output
    #input_match = re.search(input_pattern, text)
    output_match = re.search(output_pattern, text)

    #if input_match and output_match:
    if output_match:
        #input_value = input_match.group(1).strip()
        output_value = output_match.group(1).strip()
        return output_value
    else:
        return  None


In [ ]:
# Example data
from sympy import E, N
import re

data = [('4e^((3) + 3)', 4.0/E**6)]
from sympy import sympify

# Data to process
data = [('4e^((3) + 3)', '4.0')]

# Function to convert the second element of each tuple to a symbolic expression
def convert_to_symbolic(data):
    processed_data = []
    for item in data:
        # Replace '/' with '*' in the second element
        symbolic_expr = sympify(item[1].replace('/', '*'))
        processed_data.append((item[0], symbolic_expr))
    return processed_data

from sympy import sympify, N

# Function to evaluate expressions
def Evaluate_Exp(data):
    # Iterate through the list of tuples
    for equation, result in data:
        result = str(result)  # Ensure result is treated as a string
        # Check if the second element is a list
        if isinstance(result, list):
            # If it's a list and the first element is a float or int, return it directly
            if result and isinstance(result[0], (float, int)):
                return result
            else:
                return result  # Return the list as-is if not convertible to float or int
        else:
            # Attempt to convert the expression to symbolic form
            try:
                symbolic_data = convert_to_symbolic([(equation, result)])
                for eq, r in symbolic_data:
                    print("Symbolic Representation:", r)
                    evaluated_value = N(r)  # Numerically evaluate the symbolic expression
                    print("Evaluated Value:", evaluated_value)
                    return evaluated_value
            except Exception as e:
                return result

# Example usage
data = [('4e^((3) + 3)', 4.0/E**6)]

# def convert_to_symbolic(data):
#     from sympy import sympify
#     processed_data = []
#     for item in data:
#         symbolic_expr = sympify(item[1].replace('/', '*'))
#         processed_data.append((item[0], symbolic_expr))
#     return processed_data

result = Evaluate_Exp(data)
print("Final Output:", result)



Symbolic Representation: 4.0*exp(6)
Evaluated Value: 1613.71517397094
Final Output: 1613.71517397094


# Inference

In [ ]:
queries_list  = [
    "Find x given y = 2 and z = 4 in equation 1",
    "Find x given y = 2 in equaiton 4",
    "Solve equation 2 given x = 9",
    "evaluate for z in equation 7 Given x = 2, y = 4",
    "Evaluate for x = 3 in equation 5",
    "Find x, given z = 2, y = 6 in equation 5",
    "In equation 7 find y when x = 9 and z = 16",
    "Find z as x = 2 and y = 2 in equation 7",
    "Solve when x = 2 in equation 6",
    "Solve equation 1",
    "Solve equaiton 7",
    "Find e given delta = 2 in equation 1",
    "find L a = 2, k = 5, pi = 3.14, d = 3 in equation 2",
    "Find z when omega = 2 and tau = 6 in equation 3",
    "Solve for y = 3 in equation 5",
    "Find Phi for y = 7, k = 6 and const = 2 in equaiton 1",
    "Find alpha when z = 2 in equation 2",
    "Find r given k = 4 and beta = 22 in equation 8",
    "Solve for n = 2, p = 4 in equaiton 6",
    "Find alpha given v = 4 from equation 8",
    "Evaluate for Phi given that u = 7 in equation 3",
    "In equation 3, find tilde Psi given v = 3 and Psi = 2",
    "solve equation 4 for z = 8",
    "Evaluate equation 2 for x = 2",
    "Evaluate equation 4 for x = 4, y = -3",
    "Solve for z given y = 2 and x = 2 in equation 1",
    "find eta in equation 8 Given L = 6, N = 8"
]



latex_equations = [
    'x + 2y - \\sqrt{z} = 9',
    "x^2 + \\frac{1}{y} = 5",
    "e^{-3 \\sqrt{x}}",
    "x^3 + \\sqrt{y} = z",
    "4e^{x + 3}",
    "y^3 + z = 2x",
    "x + y + z^2 = 7",
    "x^3 + y^2 + z = 10",
    "\\log_{10}(x^2 + 1)",
    "\\lim_{x \\to 3} \\frac{tan(13x)}",
    "\\lim_{x \\to 2} \\frac{12 - 409.6}{x - 2}",
    "\\epsilon = - \\frac { \\Delta + 2 } { \\Delta }",
    "L = \\frac { 1 } { 4 \\pi } \\alpha K d \\alpha",
    "z = \\frac { \\omega \\tau } { 2 }",
    "2e^4y",
    "\\Phi = - \\frac { 2 Y } { k } + c o n s t",
    "\\alpha _ { 2 } = \\sqrt { 2 }",
    "r = \\frac { \\sqrt { 4 \\pi } } { \\beta } k",
    "\\lparen n + \\frac { p } { 2 }",
    "\\alpha = \\sqrt { \\frac { 1 + v } { 2 - v } }",
    "\\Phi = 2 \\sqrt { 3 } U",
    "\\tilde { \\Psi } = \\lparen \\cosh V \\rparen \\Psi",
    "2e^{-\\sqrt{z}}",
    "4e^{-\\frac{x}{2}}",
    "4e^{x^2 + 2y} + 2y",
    "x + y + z + \\sqrt{xy} = 5",
    "\\eta = \\sqrt { \\frac { L } { N } }"
]


In [ ]:
# Example usage

resultss = []
for query, latex_equation in zip(queries_list, latex_equations):
  query = query
  equation = Cleaning_equation(latex_equation)
  #print(query)
  #print(equation)

  Math_format = Latex2Math(query, equation)
  LLM_output = extractoutput_Latex2Math(Math_format)
  print('LLM output', LLM_output)

  LLM_list = ast.literal_eval(LLM_output)

  # Extract the query and equation
  LLM_query = LLM_list[0]   # The first element
  LLM_equation = LLM_list[1]  # The second element

  # Output the results
  print("LLM_query:", LLM_query)
  print("LLM_equation:", LLM_equation)

  LLM_query = str(LLM_query)
  LLM_equation = str(LLM_equation)

  query_variables = parse_query_variables(LLM_query)
  print("Extracted Variables:", query_variables)
  # Substitute values into the equation
  exprr = insert_values(LLM_equation, query_variables)
  print(exprr)
  exp_list = [str(exprr)]

  result = solve_equations(exp_list)
  print("Solved result:", result)
  answer = Evaluate_Exp(result)
  print("Final Answer:", answer)
  print("-------------------------------------------")
  resultss.append(exprr)

LLM output ['Find x given y = 2 and z = 4 in equation 1', 'x + (2)*(y) - sqrt(z) = 9']
LLM_query: Find x given y = 2 and z = 4 in equation 1
LLM_equation: x + (2)*(y) - sqrt(z) = 9
Extracted Variables: {'y': 2, 'z': 4}
x + (2)*(2) - sqrt(4) = 9
Solved result: [('x + (2)*(2) - sqrt(4) = 9', [7.00000000000000])]
Symbolic Representation: [7.00000000000000]
Final Answer: [7.00000000000000]
-------------------------------------------
LLM output ['Find x given y = 2 in equaiton 4', 'x^2 + (1)/(y) = 5']
LLM_query: Find x given y = 2 in equaiton 4
LLM_equation: x^2 + (1)/(y) = 5
Extracted Variables: {'y': 2}
x^2 + (1)/(2) = 5
Solved result: [('x^2 + (1)/(2) = 5', [-2.12132034355964, 2.12132034355964])]
Symbolic Representation: [-2.12132034355964, 2.12132034355964]
Final Answer: [-2.12132034355964, 2.12132034355964]
-------------------------------------------
LLM output ['Solve equation 2 given x = 9', 'e^(-3 sqrt(x))']
LLM_query: Solve equation 2 given x = 9
LLM_equation: e^(-3 sqrt(x))
Extrac

In [ ]:
import re

# Function to convert the expression '4.0/E**6' to '4.0*E**6'
def convert_expression_in_data(data):
    # Define the pattern to match 'number/E**something' and replace it with 'number*E**something'
    pattern = r'(\d+(\.\d+)?)/E\*\*(.*?)(?=\s|$)'  # Matches numbers like '4.0/E**6' or '6/E**-7'

    # Iterate over the list of tuples and apply the conversion
    converted_data = []
    for item in data:
        # Convert the second element to string if it's an expression
        item_str = str(item[1])  # Convert the second element to string

        # Now apply the regex substitution
        converted_item = re.sub(pattern, r'\1*E**\3', item_str)
        converted_data.append((item[0], converted_item))

    return converted_data

# Example usage
data = [('4e^((3) + 3)', 4.0/E**6)]  # Using an actual mathematical expression
converted_data = convert_expression_in_data(data)

print("Original data:", data)
print("Converted data:", converted_data)


NameError: name 'E' is not defined

In [ ]:
import re

# Function to remove the expression '4.0*exp(-6)'
def remove_expression(text):
    # Define the pattern to match '4.0*exp(-6)'
    pattern = r'4.0\*exp\(-6\)'

    # Use re.sub() to replace the pattern with an empty string
    cleaned_text = re.sub(pattern, '', text)

    return cleaned_text

# Example usage
input_text = "This is an example with the expression 4.0*exp(-6) that we want to remove."
output_text = remove_expression(input_text)

print("Original text:", input_text)
print("Cleaned text:", output_text)


Original text: This is an example with the expression 4.0*exp(-6) that we want to remove.
Cleaned text: This is an example with the expression  that we want to remove.


Could not parse or solve the equation '2*sp.E**(-sp.sqrt(7))'. Error: 'Symbol' object has no attribute 'E'
Could not parse or solve the equation 'sp.E**((9**2) + (2*8)) + (2*8)'. Error: 'Symbol' object has no attribute 'E'
Could not parse or solve the equation 'z = ((sp.E**2))/(2*(4**2))'. Error: 'Symbol' object has no attribute 'E'
